## Demo of fmm Python API

In [32]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT
from fmm import UBODTGenAlgorithm
#! pip2 install pyarrow
import pyarrow
import pyarrow.parquet as pq
import os
import pandas as pd
import ast
import numpy as np
#pd.options.mode.chained_assignment = None

In [6]:

def getPoints(file_path):
    if os.path.isfile(file_path):
        text_file = open(file_path, "r")
        wkt = text_file.read()
        text_file.close()
    return wkt
def save_results(file_base_name, cpath, opath, indices, m_wkt, e_wkt, distance):
  text_file = open(file_base_name + '-cpath.txt', "w")
  text_file.write(cpath)
  text_file.close()
  text_file = open(file_base_name + '-opath.txt', "w")
  text_file.write(opath)
  text_file.close()
  text_file = open(file_base_name + '-indices.txt', "w")
  text_file.write(indices)
  text_file.close()
  text_file = open(file_base_name + '-m_wkt.txt', "w")
  text_file.write(m_wkt)
  text_file.close()
  text_file = open(file_base_name + '-e_wkt.txt', "w")
  text_file.write(e_wkt)
  text_file.close()
  text_file = open(file_base_name + '-distance.txt', "w")
  text_file.write(distance)
  text_file.close()



### Load network data and graph

In [7]:
#network = Network("data/edges.shp")
# network = Network('../data/GPS.nosync/sonoma/edges.shp')
# print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
# print(network)
# graph = NetworkGraph(network)
network = Network("../data/GPS.nosync/sonoma/edges.shp","fid", "u", "v")
print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
graph = NetworkGraph(network)

Nodes 19594 edges 47319


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [8]:
ubodt_gen = UBODTGenAlgorithm(network,graph)

In [6]:
#status = ubodt_gen.generate_ubodt("../data/GPS.nosync/_street_nodes_dual.txt", 4, binary=False, use_omp=True)
#print status

Status: success
Time takes 1965.51 seconds



### Load UBODT data

In [9]:
ubodt = UBODT.read_ubodt_csv("../data/GPS.nosync/sonoma/ubodt.txt")

### Create FMM model

In [10]:
model = FastMapMatch(network,graph,ubodt)

### Define FMM configuration

In [11]:
# k = 50
# radius = 100
# gps_error = 50
# k = 64
# radius = 10
# gps_error = 50
k = 30
radius = 2
gps_error = 2
fmm_config = FastMapMatchConfig(k,radius,gps_error)

### Run map matching

#### Match a single trajectory

In [13]:
wkt = "LINESTRING(0.200812146892656 2.14088983050848,1.44262005649717 2.14879943502825,3.06408898305084 2.16066384180791,3.06408898305084 2.7103813559322,3.70872175141242 2.97930790960452,4.11606638418078 2.62337570621469)"

In [12]:
def get_sub_result(input_path, output_path, index):
  input_source = input_path + str(index) + '-points.txt'
  out_dir = output_path + 'sub-result-' + str(index)
  if (os.path.isfile(input_source)) & (not os.path.isdir(out_dir)):
    wkt = getPoints(input_source)
    result = model.match_wkt(wkt,fmm_config)
    if len(list(result.cpath)) > 0:
      #dir = output_path + 'sub-result-' + str(index)
      os.mkdir(out_dir)
      output_base = out_dir + '/' + str(index)
      save_results(output_base,str(list(result.cpath)), str(list(result.opath)),str(list(result.indices)),result.mgeom.export_wkt(),result.pgeom.export_wkt())
      print('index %d sucess!' % index)
    else:
      print('index %d failed!' % index)
    return result



In [13]:
RATIO = 2.23693629

In [125]:
def get_sub_result_from_df(df, output_path, index, start_time, end_time, mode, min_count, max_count):
  print(int(df.loc[index]['count']))
  if ((int(df.loc[index]['count']) >= min_count) & (int(df.loc[index]['count']) <= max_count)):
    flag = False
    if mode == 0:
      flag = (bool(df.loc[index]['is_process']) == False) & (bool(df.loc[index]['is_success']) == False)
    if mode == 1:
      flag = (bool(df.loc[index]['is_process']) == True) & (bool(df.loc[index]['is_success']) == False)
    if mode == 2:
      flag = True
    if mode == 3:
      flag = (bool(df.loc[index]['is_process']) == False) & (bool(df.loc[index]['is_success']) == False) & (pd.Timestamp(df.loc[index]['start_time']) >= start_time) & (pd.Timestamp(df.loc[index]['end_time']) < end_time)
    if mode == 4:
      flag = (bool(df.loc[index]['is_process']) == True) & (bool(df.loc[index]['is_success']) == False) & (pd.Timestamp(df.loc[index]['start_time']) >= start_time) & (pd.Timestamp(df.loc[index]['end_time']) < end_time)
    if mode == 5:
      flag = (pd.Timestamp(df.loc[index]['start_time']) >= start_time) & (pd.Timestamp(df.loc[index]['end_time']) < end_time)
    if flag:
      #df.loc[index]['is_process'] = True
      df.at[index,'is_process'] = True
      wkt = df.loc[index]['res']
      result = model.match_wkt(wkt,fmm_config)
      if len(list(result.cpath)) > 1:
        #dir = output_path + 'sub-result-' + str(index)
        n = len(result.candidates)
        distance = []
        for i in range(1, n):
          distance.append(result.candidates[i].spdist * 110000)
        df.at[index,'is_speed_calculated'] = False
        df.at[index,'distance'] = distance
        df.at[index,'mgeom'] = result.mgeom.export_wkt()
        df.at[index,'pgeom'] = result.pgeom.export_wkt()
        df.at[index, 'indices'] = list(result.indices)
        #df.at[index, 'indices'] = np.array(result.indices,dtype=int)
        df.at[index, 'cpath'] = list(result.cpath)
        if n == int(df.loc[index]['count']):
          print('Calculating speeds!')
          speeds = [i / j * RATIO for i, j in zip(distance, ast.literal_eval(df.loc[index]['time']))]
          df.at[index, 'speed'] = speeds
          df.at[index,'is_speed_calculated'] = True
        else:
          print('The counts no matching!')
        df.at[index, 'direction'] = [0]
        out_dir = output_path + 'sub-result-' + str(index)
        os.mkdir(out_dir)
        #output_base = out_dir + '/' + str(index)
        #save_results(output_base,str(list(result.cpath)), str(list(result.opath)),str(list(result.indices)),result.mgeom.export_wkt(),result.pgeom.export_wkt(),str(distance))
        df.at[index,'is_success'] = True
        print('index %d success!' % index)
        return result
      else:
        df.at[index,'is_success'] = False
        print('index %d failed!' % index)

In [21]:
res = [1, 2, 3, 4]
str(res)

'[1, 2, 3, 4]'

In [ ]:
df = pd.read_csv('../data/GPS.nosync/Sonoma-2631-new-points/out3.csv')
n = len(df)


In [107]:
# df['cpath'] = [[0]] * n
# df['indices'] = [[0]] * n
# df['direction'] = [[0]] * n
# df['speed'] = [[0]] * n
# df['distance'] = [[0]] * n
# df['direction'] = [[0]] * n
# df['indices'] = df['indices'].astype(object)
# df['cpath'] = df['cpath'].astype(object)

In [ ]:
for index in [8365, 9778, 11242]:
  get_sub_result_from_df(df, '../data/GPS.nosync/output/', index,0, 0, 2, 10, 10000)

In [ ]:
t1 = pd.Timestamp(year = 2019,  month = 10, day = 25,
           hour = 21, minute = 59, second = 59, tz = 'US/Pacific')
t2 = pd.Timestamp(year = 2019,  month = 10, day = 26,
           hour = 6, minute = 0, second = 0, tz = 'US/Pacific')
t3 = pd.Timestamp(year = 2019,  month = 10, day = 26,
           hour = 12, minute = 0, second = 0, tz = 'US/Pacific')
t4 = pd.Timestamp(year = 2019,  month = 10, day = 26,
           hour = 21, minute = 59, second = 59, tz = 'US/Pacific')
t5 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 6, minute = 0, second = 0, tz = 'US/Pacific')
t6 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 12, minute = 0, second = 0, tz = 'US/Pacific')
t7 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 21, minute = 59, second = 59, tz = 'US/Pacific')
t8 = pd.Timestamp(year = 2019,  month = 10, day = 28,
           hour = 6, minute = 0, second = 0, tz = 'US/Pacific')
n = len(df) 
for index in range(1, n - 1):
  get_sub_result_from_df(df, '../data/GPS.nosync/output/26-06-26-12/', index,t2, t3, 5, 5, 10000000)
# for index in range(1, n - 1):
#   get_sub_result_from_df(df, '../data/GPS.nosync/output/26-12-26-22/', index,t3, t4, 5, 5, 10000000)
# for index in range(1, n - 1):
#   get_sub_result_from_df(df, '../data/GPS.nosync/output/26-22-27-06/', index,t4, t5, 5, 5, 10000000)
# for index in range(1, n - 1):
#   get_sub_result_from_df(df, '../data/GPS.nosync/output/27-06-27-12/', index,t5, t6, 5, 5, 10000000)
# for index in range(1, n - 1):
#   get_sub_result_from_df(df, '../data/GPS.nosync/output/27-12-27-22/', index,t6, t7, 5, 5, 10000000)


In [ ]:
for index in range(500, n):
  get_sub_result_from_df(df, '../data/GPS.nosync/output/', index, 0, 0, 2, 10, 10000)

In [144]:
for i in range(0, n):
    if len(df.loc[i]['speed']) > 1:
        df.at[i,'is_speed_calculated'] = True

In [145]:
df_s = df[df['is_speed_calculated'] == True]
len(df_s)

11884

In [155]:
df_s.iloc[0]

Unnamed: 0                                                             1
index                                                                  1
caid                   000240069cae6c8106ed10b34ac23e734f7e156a9e170d...
start_time                                     2019-10-27 03:20:17-07:00
end_time                                       2019-10-27 12:04:15-07:00
count                                                                 61
is_process                                                          True
is_success                                                          True
res                    LINESTRING(-122.70838 38.347546,-122.70817 38....
time                   [23.0, 11.0, 35.0, 21.0, 2.0, 38.0, 50.0, 27.0...
distance               [1.62712969297, 90.8758931912, 0.0, 0.0, 0.0, ...
speed                  [0.158251541684, 18.4803257605, 0.0, 0.0, 0.0,...
direction                                                            [0]
cpath                  [22615, 26161, 13262, 288, 2

In [156]:
for i in range(0, 500):
    
    #print("Speeds size: %d, indices size: %d" % (len(df_s.iloc[i]['speed']), len(df_s.iloc[i]['indices'])))

Speeds size: 60, indices size: 61
Speeds size: 193, indices size: 194
Speeds size: 82, indices size: 83
Speeds size: 9, indices size: 10
Speeds size: 334, indices size: 335
Speeds size: 125, indices size: 126
Speeds size: 905, indices size: 906
Speeds size: 1245, indices size: 1246
Speeds size: 10, indices size: 11
Speeds size: 442, indices size: 443
Speeds size: 888, indices size: 889
Speeds size: 15, indices size: 16
Speeds size: 145, indices size: 146
Speeds size: 212, indices size: 213
Speeds size: 908, indices size: 909
Speeds size: 657, indices size: 658
Speeds size: 34, indices size: 35
Speeds size: 378, indices size: 379
Speeds size: 303, indices size: 304
Speeds size: 865, indices size: 866
Speeds size: 310, indices size: 311
Speeds size: 13, indices size: 14
Speeds size: 897, indices size: 898
Speeds size: 121, indices size: 122
Speeds size: 848, indices size: 849
Speeds size: 107, indices size: 108
Speeds size: 24, indices size: 25
Speeds size: 189, indices size: 190
Speeds 

In [39]:
result = get_sub_result_from_df(df, '../data/GPS.nosync/output/', 13,0, 0, 2, 10, 10000)
# for index in range(1, n - 1):
#print "Matched path: ", list(result.cpath)
#wkt = getPoints('data/GPS.nosync/output/00074014459977889ff22e04d309cbc68fb1720990d2d6a7251eb6c17fbf5f72-points.txt')

126
Calculating speeds!
index 13 success!


In [146]:
df.to_csv('../data/GPS.nosync/Sonoma-2631-new-points/speeds_info.csv')

In [ ]:
save_results('data/GPS.nosync/output/0-00074',str(list(result.cpath)), str(list(result.opath)),str(list(result.indices)),result.mgeom.export_wkt(),result.pgeom.export_wkt())

##Print detailed match information

In [ ]:
candidates = []
for c in result.candidates:
    candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

In [ ]:
import pandas as pd
df = pd.DataFrame(candidates,
                  columns=["eid","source","target","error","length","offset","spdist","ep","tp"])

In [ ]:
df.head()

#### Match trajectories in a GPS file

In [ ]:
from fmm import GPSConfig,ResultConfig

In [ ]:
input_config = GPSConfig()
input_config.file = "../data/trips.csv"
input_config.id = "id"

In [ ]:
print input_config.to_string()

In [ ]:
result_config = ResultConfig()
result_config.file = "../data/mr.txt"
result_config.output_config.write_opath = True
print result_config.to_string()

In [ ]:
status = model.match_gps_file(input_config, result_config, fmm_config)

In [ ]:
print status

In [ ]:


filename = "../data/in_zone_grouped_evacuation_stops.parquet" 



df = pq.read_table(source=filename).to_pandas()

In [ ]:
print(df.loc[100])